In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from linkage_vis import LinkageViewer
from bending_validation import suppress_stdout

l = elastic_rods.RodLinkage('../../examples/hyperboloid_open.obj', 5)
mat = elastic_rods.RodMaterial('+', 20000, 0.3, [0.05, 0.05, 0.001, 0.001])
l.setMaterial(mat)
driver=16

view = LinkageViewer(l, width=768)
view.setCameraParams(((0.21363949107371888, -1.5745838914291204, 3.221274247846029),
 (-0.39958918133000937, -0.9163352741275177, -0.02565446459509408),
 (-0.395065608121624, 0.38240522741182065, 0.06716453768425666)))
view.show()

In [ ]:
opts = elastic_rods.NewtonOptimizerOptions()
opts.useIdentityMetric=True
with suppress_stdout(): elastic_rods.restlen_solve(l, opts)
view.update()
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
opts.useIdentityMetric = False
with suppress_stdout(): elastic_rods.compute_equilibrium(l, options=opts, fixedVars=fixedVars)
view.update()

In [ ]:
restLinkage = elastic_rods.RodLinkage(l)

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.useIdentityMetric = True
    opts.useNegativeCurvatureDirection = True
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

In [ ]:
# cr = open_linkage(l, driver, -4.9 * np.pi / 16, 15, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=True, useTargetAngleConstraint=False);
report = open_linkage(l, driver, -4 * np.pi / 16, 10, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=True, useTargetAngleConstraint=True);

In [ ]:
copyLinkage = elastic_rods.RodLinkage(l)

In [ ]:
l = elastic_rods.RodLinkage(restLinkage)
cr = open_linkage(l, driver, -4 * np.pi / 16, 15, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=True, useTargetAngleConstraint=True, earlyStopIt=8);
cr = open_linkage(l, driver, -4 * np.pi / 16, 15, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=4, verbose=True, useTargetAngleConstraint=True, earlyStopIt=2);

In [ ]:
from compute_vibrational_modes import *
from mode_viewer import ModeViewer
lambdas, modes = compute_vibrational_modes(l, fixedVars[:-1], MassMatrixType.FULL, n=4);
mview = ModeViewer(l, modes, lambdas, width=1024, height=768, amplitude=0.2)
mview.show()

In [ ]:
# For debugging Newton step:
# l.updateSourceFrame()
# l.setDoFs(l.getDoFs())
optimizer = elastic_rods.get_equilibrium_optimizer(l, targetAverageAngle=0.7975787788597704, fixedVars=list(range(jdo, jdo + 6)))
optimizer.options.beta = 1e-8
optimizer.options.useIdentityMetric = True
prob = optimizer.get_problem()
prob.setVars(prob.applyBoundConstraints(prob.getVars()))
d = optimizer.newton_step(False)

In [ ]:
elastic_rods.linkage_deformation_analysis(restLinkage, l, "hyperboloid_deformation_analysis.msh")

In [ ]:
restLinkage.writeTriangulation('test.msh')